In [1]:
import os

os.chdir('..')

In [2]:
os.getcwd()

'/workspace/gan/3D-FM-GAN'

In [3]:
print(__package__)  

None


In [32]:
from accelerate import Accelerator

In [34]:
accelerator = Accelerator(mixed_precision='fp16')

In [45]:
from model.stylegan2.model import StyledConv, ConstantInput
import torch
import torch.nn as nn

In [38]:
channel_multiplier=2
style_dim = 512
channels = {
            4: 512,
            8: 512,
            16: 512,
            32: 512,
            64: 256 * channel_multiplier,
            128: 128 * channel_multiplier,
            256: 64 * channel_multiplier,
            512: 32 * channel_multiplier,
            1024: 16 * channel_multiplier,
        }
blur_kernel=[1, 3, 3, 1]


style= torch.randn(10, 14, 512).to('cuda', dtype=torch.float16)
enc_t_result = torch.randn(10, 512).to('cuda', dtype=torch.float16)
input = ConstantInput(channels[4]).to('cuda')
conv = StyledConv(channels[4], channels[4], 3, style_dim, blur_kernel=blur_kernel).to('cuda')

In [53]:
input.input.dtype

torch.float32

In [39]:
style.dtype

torch.float16

In [44]:
accelerator.mixed_precision

'fp16'

In [49]:
with accelerator.autocast():
    conv_inp = input(enc_t_result)
    print(conv_inp.dtype)
    output = conv(conv_inp, style[:, 0] ,None)
    print(output.dtype)

torch.float32
torch.float32


In [ ]:
# ConstantInput 여기가 fp32을 내놓는다.
# 먼저 생성된 self.parameter를 이용하기 떄문

In [51]:
with torch.cuda.amp.autocast():
    print(enc_t_result.dtype)
    conv_inp = input(enc_t_result)
    print(conv_inp.dtype)
    output = conv(input(enc_t_result), style[:, 0] ,None)
    print(output.dtype)

torch.float16
torch.float32
torch.float32


In [47]:
test_inp = torch.randn(10, 20).to('cuda')
lin = nn.Linear(20, 30).to('cuda')
with torch.cuda.amp.autocast():
    output = lin(test_inp)
    print(output.dtype)

torch.float16


In [54]:
# autocast의 적용범위가 어디까지인지?
# StyleGAN-ADA는 어떻게 된건지?
# 모든 것에 적용되는게 아닌건가?
input = ConstantInput(channels[4])
with torch.autocast():
    out = input.cuda()(enc_t_result)
    print(out.dtype)

AttributeError: module 'torch' has no attribute 'autocast'

In [56]:
from torch import autocast

ImportError: cannot import name 'autocast' from 'torch' (/opt/conda/lib/python3.8/site-packages/torch/__init__.py)

In [55]:
torch.__version__

'1.10.0a0+ecc3718'

In [1]:
from torch.cuda import amp

# scaler = amp.grad_scaler.GradScaler()

In [2]:
amp.GradScaler()